In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [ ]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self, decoder).__init__()
        self.decoder = decoder
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        for x in range(5):
            x = self.decoder(x)
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.decoder(x)
        return x